# Function Calling with Gemini 2.5

In [1]:
from google import genai
client = genai.Client()

In [2]:
import requests,json
def get_current_weather(city:str)->dict:
    """ can be used to get/fetch current weather information for a city name
    """
    api_key = "6a8b0ac166a37e2b7a38e64416b3c3fe"

    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}"
    response = requests.get(url)
    response = response.content.decode()
    response = json.loads(response)
    output = {"City Name":city,"weather":response["weather"][0]['description'],
              "temperature":response['main']['temp'],
              "unit":"kelvin"}

    return output

In [3]:
get_current_weather("mumbai")

{'City Name': 'mumbai',
 'weather': 'broken clouds',
 'temperature': 299.93,
 'unit': 'kelvin'}

In [4]:
get_current_weather("delhi")

{'City Name': 'delhi',
 'weather': 'few clouds',
 'temperature': 308.23,
 'unit': 'kelvin'}

### Tool Integration with LLM

- metadata for the tool
    - metdata needs to include 3 compoents
        - name of function
        - description of function
        - arguments: args and its description
    - OPEN API Format to define metadata

In [27]:
tools_def = [{
    "name":"get_current_weather",
    "description":"this function is used to get/fetch current weather information for any given city",
    "parameters":{"type":"object",
                  "properties":{"city":{"type":"string","description":"name of any location/city e.g. mumbai, new york"}},
                  "required":["city"],},
                  
},]

In [ ]:
from google.genai import types
import json
tools = types.Tool(function_declarations=tools_def)
config = types.GenerateContentConfig(tools=[tools],
                                     automatic_function_calling=types.FunctionCallingConfig(mode='AUTO'),
                                     system_instruction="You are a helpful assistant, you need to answer user question, use tools when necessary and answer question based on your ability provide final response with answer to all queries asked.",
                                     temperature=0.2,seed=5) #NONE, ANY

tool_map = {"get_current_weather":get_current_weather}


In [34]:
def generate_response(prompt:str):
    contents = [types.Content(role='user',parts=[types.Part(text=prompt)])]

    first_response=client.models.generate_content(model='gemini-2.0-flash',
                                                  contents=contents,
                                                  config=config)
    #print(first_response)
    if first_response.candidates[0].content.parts[0].function_call:
        # do something
        print("LLM decided to make a function call,", first_response.candidates[0].content.parts[0].function_call)
        tool_call = first_response.candidates[0].content.parts[0].function_call
        contents.append(first_response.candidates[0].content)
        
        tool_name = tool_call.name
        tool_args = tool_call.args
        function_to_ex = tool_map[tool_name]
        tool_output = function_to_ex(**tool_args)

        tool_resp = types.Part.from_function_response(name=tool_name,response={"result":tool_output})

        contents.append(types.Content(role='user',parts=[tool_resp]))

        second_response = client.models.generate_content(model='gemini-2.0-flash',
                                                         contents=contents)
        return second_response.text
        
    else:
        return first_response.text

In [35]:
generate_response("define quantum computing?")

'I am not equipped to provide a definition of quantum computing.\n'

In [36]:
generate_response("what is the current weather in delhi?")

LLM decided to make a function call, id=None args={'city': 'delhi'} name='get_current_weather'


'The current weather in Delhi is few clouds with a temperature of 308.23 Kelvin.\n'

In [37]:
result = generate_response("Tell me something about city Delhi and what is the current weather in delhi?")
print(result)

LLM decided to make a function call, id=None args={'city': 'delhi'} name='get_current_weather'
Okay, here's some information about Delhi and its current weather:

**About Delhi:**

*   **Historical Significance:** Delhi is a city steeped in history, serving as the capital of various empires and kingdoms throughout centuries. This rich past is evident in its numerous historical monuments, including the Red Fort, Qutub Minar, Humayun's Tomb, and India Gate.
*   **Capital of India:** Delhi serves as the capital of India, housing important government buildings, embassies, and administrative centers.
*   **Cultural Hub:** Delhi is a vibrant cultural center, blending diverse traditions, languages, and cuisines. It offers a mix of ancient and modern influences, visible in its architecture, arts, and lifestyle.
*   **Metropolitan City:** Delhi is a bustling metropolitan city with a large population and a fast-paced lifestyle. It's a major economic hub, attracting people from all over India for

## Integrating Multiple Tools


In [11]:
!python -m pip install wikipedia --quiet

In [12]:
import wikipedia

def get_wkipedia_summary(query:str)->str:
    response = wikipedia.summary(query)
    return response

In [13]:
print(get_wkipedia_summary("capital city of Sikkim"))

Gangtok (Nepali: gāntok, pronounced [gantok], Sikkimese: [ɡaŋt̪ʰòk]) is the capital and the most populous city of the Indian state of Sikkim. The seat of eponymous district, Gangtok is in the eastern Himalayan range, at an elevation of 1,650 m (5,410 ft). The city's population of 100,000 consists of the three Sikkimese ethnicities: the Bhutias, Lepchas, Indian Gorkhas as well as plainsmen from other states of India. Within the higher peaks of the Himalayas and with a year-round mild temperate climate, Gangtok is at the centre of Sikkim's tourism industry.
Gangtok rose to prominence as a popular Buddhist pilgrimage site after the construction of the Enchey Monastery in 1840. In 1894, the ruling Sikkimese Chogyal, Thutob Namgyal, transferred the capital to Gangtok. In the early 20th century, Gangtok became a major stopover on the trade route between Lhasa in Tibet and cities such as Kolkata (then Calcutta) in British India. After India won its independence from the British Empire in 1947

In [14]:
tools_def = [{
    "name":"get_current_weather",
    "description":"this function is used to get/fetch current weather information for any given city",
    "parameters":{"type":"object",
                  "properties":{"city":{"type":"string","description":"name of any location/city e.g. mumbai, new york"}},
                  "required":["city"],},
                  
},
{
    "name":"get_wikipedia_summary",
    "description":"this function is used to get information about locaiton, places, people, history, from wkipedia, given the topic",
    "parameters":{"type":"object",
                  "properties":{"query":{"type":"string","description":"topic to search on wikipedia e.g. capital of India"}},
                  "required":["query"],},
                  
},]


In [15]:
from google.genai import types
import json
tools = types.Tool(function_declarations=tools_def)
config = types.GenerateContentConfig(tools=[tools],
                                     automatic_function_calling=types.FunctionCallingConfig(mode='AUTO')) #NONE, ANY

tool_map = {"get_current_weather":get_current_weather,
            "get_wikipedia_summary":get_wkipedia_summary}

In [19]:
def generate_response(prompt:str):
    contents = [types.Content(role='user',parts=[types.Part(text=prompt)])]

    while True:

        response=client.models.generate_content(model='gemini-2.0-flash', contents=contents, config=config)
        #print(first_response)
        if response.candidates[0].content.parts[0].function_call:
            # do something
            print("LLM decided to make a function call,", response.candidates[0].content.parts)
            tool_calls = response.candidates[0].content.parts
            contents.append(response.candidates[0].content)

            for tool_call in tool_calls:
                tool_name = tool_call.function_call.name
                tool_args = tool_call.function_call.args
                function_to_ex = tool_map[tool_name]
                tool_output = function_to_ex(**tool_args)

                tool_resp = types.Part.from_function_response(name=tool_name,response={"result":tool_output})

                contents.append(types.Content(role='user',parts=[tool_resp]))

        else:
            break
    return response.text

In [20]:
result = generate_response("Tell me something about city Delhi and what is the current weather in delhi?")
print(result)

LLM decided to make a function call, [Part(
  function_call=FunctionCall(
    args={
      'query': 'Delhi'
    },
    name='get_wikipedia_summary'
  )
), Part(
  function_call=FunctionCall(
    args={
      'city': 'delhi'
    },
    name='get_current_weather'
  )
)]
Delhi, in mathematics, refers to the del operator (nabla), a vector differential operator represented by ∇. It's used in vector calculus to denote the gradient, divergence, or curl of a field.

The current weather in Delhi is few clouds and the temperature is 308.23 Kelvin.



In [21]:
result = generate_response("Tell me something about city Delhi and Mumbai and what is the current weather in delhi, mumbai and bangalore?")
print(result)

LLM decided to make a function call, [Part(
  function_call=FunctionCall(
    args={
      'query': 'Delhi'
    },
    name='get_wikipedia_summary'
  )
), Part(
  function_call=FunctionCall(
    args={
      'query': 'Mumbai'
    },
    name='get_wikipedia_summary'
  )
), Part(
  function_call=FunctionCall(
    args={
      'city': 'delhi'
    },
    name='get_current_weather'
  )
), Part(
  function_call=FunctionCall(
    args={
      'city': 'mumbai'
    },
    name='get_current_weather'
  )
), Part(
  function_call=FunctionCall(
    args={
      'city': 'bangalore'
    },
    name='get_current_weather'
  )
)]
Okay, here's some information about Delhi and Mumbai, along with the current weather in Delhi, Mumbai, and Bangalore:

**Delhi:**

*   I found a Wikipedia entry for "Del" (with one "l"), which refers to an operator in mathematics, particularly vector calculus, represented by the nabla symbol (∇). It's used to denote derivatives, gradients, divergence, and curl. I can search for